In [2]:
from IPython import get_ipython
colab = 'google.colab' in str(get_ipython())

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
   
  !mkdir ./texts
  !mkdir ./bertonomics2
  !mkdir ./Bertsaved
  !cp gdrive/MyDrive/KnowledgeGraph/data/slfedkey.txt .
#  !cp -r gdrive/MyDrive/KnowledgeGraph/SAVE/entdata .
  !cp gdrive/MyDrive/KnowledgeGraph/data/t1key.txt .
  !cp gdrive/MyDrive/KnowledgeGraph/data/TB_inc.txt ./texts
  !cp gdrive/MyDrive/KnowledgeGraph/data/combinedreal.txt.zip ./texts
  !unzip ./texts/combinedreal.txt.zip
  !ls
  !cp ./combined.txt ./texts/

Mounted at /content/gdrive
Archive:  ./texts/combinedreal.txt.zip
  inflating: combined.txt            
bertonomics2  combined.txt  sample_data   t1key.txt
Bertsaved     gdrive	    slfedkey.txt  texts


In [2]:
!cp -r gdrive/MyDrive/KnowledgeGraph/SAVE/entdata/BEST_pretrained .
!ls 

bertonomics  BEST_pretrained  gdrive	   slfedkey.txt  texts
Bertsaved    combined.txt     sample_data  t1key.txt


In [12]:
!mkdir ./bertonomics2

In [3]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3 MB 6.8 MB/s 
     |████████████████████████████████| 3.1 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 66.1 MB/s 
     |████████████████████████████████| 895 kB 55.3 MB/s 
     |████████████████████████████████| 61 kB 548 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./texts/").glob("*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"
])

# Save files to disk
tokenizer.save_model("bertonomics2")

['bertonomics2/vocab.json', 'bertonomics2/merges.txt']

In [ ]:
!cp -r ./bertonomics gdrive/MyDrive/KnowledgeGraph/SAVE/entdata/

In [10]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

#!mkdir ./bertonomics

tokenizer = ByteLevelBPETokenizer('./bertonomics2/vocab.json',
    './bertonomics2/merges.txt'
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>"))
)
tokenizer.enable_truncation(max_length=512)

print(tokenizer.encode("Interest rates affect short term unemployment."))
print(tokenizer.encode("Interest rates affect short term unemployment.").tokens)
print(tokenizer.encode("Interest rates affect short term unemployment.").ids)

Exception: ignored

In [ ]:
!pip install datasets
from transformers import LineByLineTextDataset
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset
from transformers import BertForMaskedLM
from transformers import BertConfig



     |████████████████████████████████| 298 kB 23.0 MB/s 
     |████████████████████████████████| 1.1 MB 35.1 MB/s 
     |████████████████████████████████| 132 kB 45.9 MB/s 
     |████████████████████████████████| 243 kB 29.7 MB/s 
     |████████████████████████████████| 160 kB 47.9 MB/s 
     |████████████████████████████████| 192 kB 29.3 MB/s 
     |████████████████████████████████| 271 kB 47.2 MB/s 


In [ ]:
!ls 

bertonomics  combined.txt  gdrive	slfedkey.txt  texts
Bertsaved    entdata	   sample_data	t1key.txt


In [8]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
model = RobertaForMaskedLM(config=config)
#model = RobertaForMaskedLM.from_pretrained("./BEST_pretrained", output_hidden_states=True)


In [9]:
from transformers import RobertaTokenizerFast
from transformers import LineByLineTextDataset
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")#"./bertonomics", max_len=512, local_files_only=True)
#dataset = load_dataset('text', data_files="./texts/TB_inc.txt", split='train')

"""def encode(examples):
  return tokenizer(examples['text'], truncation=True, padding='max_length')

dataset = LineByLineTextDataset.map(encode, batched=True, block_size=128)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])"""


dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./combined.txt",
    block_size=128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.25
)

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Bertsaved",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=1,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [12]:
trainer.train()
trainer.save_model("./Bertsaved")
!cp -r ./Bertsaved gdrive/MyDrive/KnowledgeGraph/SAVE/entdata/

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 1323601
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 20682
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.498500
1000,5.753500
1500,5.535800
2000,5.345400
2500,5.156500
3000,4.856400
3500,4.275300
4000,3.780100
4500,3.466600
5000,3.229500


Saving model checkpoint to ./Bertsaved/checkpoint-10000
Configuration saved in ./Bertsaved/checkpoint-10000/config.json
Model weights saved in ./Bertsaved/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./Bertsaved/checkpoint-20000
Configuration saved in ./Bertsaved/checkpoint-20000/config.json
Model weights saved in ./Bertsaved/checkpoint-20000/pytorch_model.bin
Deleting older checkpoint [Bertsaved/checkpoint-10000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./Bertsaved
Configuration saved in ./Bertsaved/config.json
Model weights saved in ./Bertsaved/pytorch_model.bin


##Demonstration of Model

In [1]:
from IPython import get_ipython
colab = 'google.colab' in str(get_ipython())

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
  
!cp -r gdrive/MyDrive/KnowledgeGraph/SAVE/entdata/BEST_pretrained .
!pip install transformers


Mounted at /content/gdrive
     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 61 kB 444 kB/s 
     |████████████████████████████████| 3.3 MB 21.0 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
from transformers import RobertaTokenizerFast
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM.from_pretrained(
    "./BEST_pretrained",
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False
)

In [7]:
!cp -r gdrive/MyDrive/KnowledgeGraph/SAVE/entdata/BEST_pretrained .
!ls 

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
#tokenizer = RobertaTokenizerFast.from_pretrained("./bertonomics", do_lower_case=False)

BEST_pretrained  gdrive  sample_data


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [8]:
from transformers import pipeline

In [ ]:
!ls

bertonomics  BEST_pretrained  entdata  sample_data   t1key.txt
Bertsaved    combined.txt     gdrive   slfedkey.txt  texts


In [9]:
fill_mask = pipeline(
    "fill-mask",
    model="./BEST_pretrained",
    tokenizer=tokenizer
)

In [12]:
fill_mask("Rising <mask> rates has increased short-term unemployment.")

[{'score': 0.05894554778933525,
  'sequence': 'Rising interest rates has increased short-term unemployment.',
  'token': 773,
  'token_str': ' interest'},
 {'score': 0.050076842308044434,
  'sequence': 'Rising benefit rates has increased short-term unemployment.',
  'token': 1796,
  'token_str': ' benefit'},
 {'score': 0.049913592636585236,
  'sequence': 'Rising compensation rates has increased short-term unemployment.',
  'token': 4660,
  'token_str': ' compensation'},
 {'score': 0.04083069786429405,
  'sequence': 'Rising unemployment rates has increased short-term unemployment.',
  'token': 5755,
  'token_str': ' unemployment'},
 {'score': 0.03276224061846733,
  'sequence': 'Rising labor rates has increased short-term unemployment.',
  'token': 4178,
  'token_str': ' labor'}]

In [15]:
fill_mask("Strong performance by tech stocks is likely to change <mask>.")

[{'score': 0.12128455191850662,
  'sequence': 'Strong performance by tech stocks is likely to change slightly.',
  'token': 2829,
  'token_str': ' slightly'},
 {'score': 0.06389595568180084,
  'sequence': 'Strong performance by tech stocks is likely to change slowly.',
  'token': 5764,
  'token_str': ' slowly'},
 {'score': 0.06019546464085579,
  'sequence': 'Strong performance by tech stocks is likely to change expected.',
  'token': 421,
  'token_str': ' expected'},
 {'score': 0.04967236891388893,
  'sequence': 'Strong performance by tech stocks is likely to change further.',
  'token': 617,
  'token_str': ' further'},
 {'score': 0.04721638560295105,
  'sequence': 'Strong performance by tech stocks is likely to change little.',
  'token': 410,
  'token_str': ' little'}]